In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mech-interp-data/cpp_numbered_qa.json
/kaggle/input/mech-interp-data/python_numbered_qa.json
/kaggle/input/mech-interp-data/numbered_qa_strings.json


# 1. Setup and Loading the Model

In [2]:
# Part 1: Setup, Dependency Installation, and Data Loading

# Step 1: Install required libraries quietly
!pip install -q transformers datasets peft bitsandbytes accelerate

# Step 2: Import necessary modules
import os
import json
import copy
import pandas as pd
import torch
from kaggle_secrets import UserSecretsClient
from datasets import Dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
)
from peft import get_peft_model, LoraConfig

# Step 3: Set up Hugging Face authentication
try:
    user_secrets = UserSecretsClient()
    hf_token = user_secrets.get_secret("3b_token")
    os.environ["HF_TOKEN"] = hf_token
    print("Hugging Face token set successfully.")
except Exception as e:
    print(f"Could not retrieve Hugging Face token. Please ensure the 'aloo' secret is set. Error: {e}")
    # Fallback for local testing if secrets aren't available
    hf_token = None

# Step 4: Define model and load model and tokenizer
model_id = "meta-llama/Llama-3.2-3B"

tokenizer = AutoTokenizer.from_pretrained(model_id)
# Set a pad token if it doesn't exist
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map="auto",
)
print("Model and tokenizer loaded.")





   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 26.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 70.1 MB/s eta 0:00:00:00:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 

2025-06-08 04:33:36.248268: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749357216.422690      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749357216.476313      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Hugging Face token set successfully.


tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

Model and tokenizer loaded.


# 2. Loading the data

In [3]:
# Step 5: Load and process the JSON dataset
# Part 1: Setup and Data Loading (Concise Version)

# Step 1: Install required libraries quietly
!pip install -q transformers datasets peft bitsandbytes accelerate

# Step 2: Import necessary modules
import os
import json
import copy
import pandas as pd
import torch
from kaggle_secrets import UserSecretsClient
from datasets import Dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
)
from peft import get_peft_model, LoraConfig

# Step 3: Set up Hugging Face authentication
try:
    user_secrets = UserSecretsClient()
    hf_token = user_secrets.get_secret("3b_token")
    os.environ["HF_TOKEN"] = hf_token
    print("Hugging Face token set successfully.")
except Exception as e:
    print(f"Could not retrieve Hugging Face token. Please ensure the 'aloo' secret is set. Error: {e}")
    hf_token = None

# Step 4: Define and load model and tokenizer
model_id = "meta-llama/Llama-3.2-3B"

tokenizer = AutoTokenizer.from_pretrained(model_id)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map="auto",
)
print("Model and tokenizer loaded.")

# Step 5: Load and process the JSON dataset (Concise Method)
json_path = "/kaggle/input/mech-interp-data/numbered_qa_strings.json"

with open(json_path, 'r') as f:
    raw_data = json.load(f)

# This assumes the JSON is a flat dictionary like {"1": "text1", "2": "text2"}
hf_dataset = Dataset.from_dict({"text": list(raw_data.values())})

# Step 6: Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        truncation=True,
        padding="max_length",
        max_length=128
    )

tokenized_dataset = hf_dataset.map(tokenize_function, batched=True, remove_columns=["text"])
print("Dataset loaded and tokenized.")
print(tokenized_dataset)

Hugging Face token set successfully.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Model and tokenizer loaded.


Map:   0%|          | 0/133799 [00:00<?, ? examples/s]

Dataset loaded and tokenized.
Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 133799
})


# 3. Model Preparation and Fine-Tuning

In [4]:
# Part 2: Model Preparation and Fine-Tuning

# Step 1: Clone the original model's state dictionary before applying LoRA
original_state_dict = copy.deepcopy(model.state_dict())
print("Original model weights have been cloned.")

# Step 2: Apply LoRA configuration using PEFT
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, lora_config)
print("\nLoRA configuration applied.")
model.print_trainable_parameters()

# Step 3: Set up Training Arguments
training_args = TrainingArguments(
    output_dir="/kaggle/working/llama3_finetuned",
    num_train_epochs=1,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    logging_steps=10,
    fp16=True, # Enable mixed-precision training
    save_strategy="no", # Do not save checkpoints
    report_to="none", # Disable W&B logging
)

# Step 4: Create the Trainer and start fine-tuning
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

print("\nStarting fine-tuning...")
trainer.train()
print("Fine-tuning complete.")

Original model weights have been cloned.


No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.



LoRA configuration applied.
trainable params: 2,293,760 || all params: 3,215,043,584 || trainable%: 0.0713

Starting fine-tuning...


Step,Training Loss
10,2.220200
20,2.097300
30,2.203400
40,2.030500
50,2.208400
60,2.011200
70,2.015100
80,2.010700
90,1.994200
100,2.032700


KeyboardInterrupt: 

# 4. Fine-Tune Model

In [ ]:
# Part 3: Weight Analysis and Reporting

print("\nStarting weight analysis...")

# Step 1: Initialize a list to store the weight change information
lora_deltas = []

# Step 2: Iterate through the named parameters of the fine-tuned model
for name, param in model.named_parameters():
    # We only care about the parameters that were trained (the LoRA adapters)
    if param.requires_grad:
        # The "delta" for a LoRA parameter is its final value,
        # as it was added to the model (initialized near zero).
        delta_tensor = param.detach().cpu().float()

        mean_delta = delta_tensor.mean().item()
        mean_abs_delta = delta_tensor.abs().mean().item()

        lora_deltas.append((name, mean_delta, mean_abs_delta))

print(f"Analyzed {len(lora_deltas)} trainable LoRA layers.")

# Step 3: Sort the layers by the mean absolute delta in descending order
lora_deltas.sort(key=lambda x: x[2], reverse=True)

# Step 4: Create a Pandas DataFrame with the top-k results
top_k = 27
df = pd.DataFrame(
    lora_deltas[:top_k],
    columns=["layer_name", "mean_change", "mean_abs_change"]
)

# Step 5: Save the DataFrame to a CSV file
output_path = "/kaggle/working/lora_weight_deltas.csv"
df.to_csv(output_path, index=False)

print(f"\nTop {top_k} LoRA weight changes saved to {output_path}")
print("---" * 10)
print(df)
